In [7]:
import pandas as pd
from pathlib import Path
import os
from datasets import Dataset

seed = 42

In [9]:
DATA = '~/data/spancat/'

In [10]:
df = pd.read_csv(DATA+'strategies-df.csv')

In [11]:
labels = [label[1:] for label in df.values.tolist()]
df['labels'] = labels

ds = Dataset.from_pandas(df[['text', 'labels']], preserve_index=False).train_test_split(test_size=0.2, seed=seed)

In [13]:
ds.save_to_disk(DATA+"strategies-ds.hf")

Saving the dataset (0/1 shards):   0%|          | 0/1084 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/271 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 1084
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 271
    })
})